In [1]:
from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.contrib.tensor_forest.python import tensor_forest
from tensorflow.python.ops import resources

C:\Users\StrikeWade\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('../mnist', one_hot=False)

Extracting ../mnist\train-images-idx3-ubyte.gz
Extracting ../mnist\train-labels-idx1-ubyte.gz
Extracting ../mnist\t10k-images-idx3-ubyte.gz
Extracting ../mnist\t10k-labels-idx1-ubyte.gz


In [3]:
#parameters
num_steps = 500 #Total steps to train
batch_size = 1024 #The number of samples per batch
num_classes = 10 #Only have 10 digits
num_features = 784 #Each image is 28x28 pixels
num_trees = 10
max_nodes = 1000

#Input and target data
x = tf.placeholder(tf.float32, shape=[None, num_features])
#For random forest, labels must be integers (the class id)
y = tf.placeholder(tf.float32, shape=[None])

#Random Forest parameters
hparams = tensor_forest.ForestHParams(num_classes=num_classes,
                                      num_features=num_features,
                                      num_trees=num_trees,
                                      max_nodes=max_nodes).fill()

In [4]:
print(hparams)

In [5]:
#Build the Random Forest
forest_graph = tensor_forest.RandomForestGraphs(hparams)
#Get training graph and loss
train_op = forest_graph.training_graph(x,y)
loss_op = forest_graph.training_loss(x,y)

#Measure the accuracy
infer_op, _, _ = forest_graph.inference_graph(x)
correct_prediction = tf.equal(tf.argmax(infer_op, 1), tf.cast(y, tf.int64))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#Initialize the variables and forest resource
init_vars = tf.group(tf.global_variables_initializer(), resources.initialize_resources(resources.shared_resources()))

INFO:tensorflow:Constructing forest with params = 
INFO:tensorflow:{'leaf_model_type': 0, 'split_type': 0, 'param_file': None, 'num_trees': 10, 'feature_bagging_fraction': 1.0, 'finish_type': 0, 'split_after_samples': 250, 'initialize_average_splits': False, 'num_outputs': 1, 'num_splits_to_consider': 28, 'early_finish_check_every_samples': 0, 'base_random_seed': 0, 'split_pruning_name': 'none', 'inference_tree_paths': False, 'model_name': 'all_dense', 'pruning_type': 0, 'bagged_num_features': 784, 'max_fertile_nodes': 0, 'bagged_features': None, 'split_name': 'less_or_equal', 'regression': False, 'dominate_method': 'bootstrap', 'num_features': 784, 'bagging_fraction': 1.0, 'dominate_fraction': 0.99, 'prune_every_samples': 0, 'valid_leaf_threshold': 1, 'stats_model_type': 0, 'num_output_columns': 11, 'max_nodes': 1000, 'num_classes': 10, 'collate_examples': False, 'split_finish_name': 'basic', 'use_running_stats_method': False, 'checkpoint_stats': False}


NotFoundError: Op type not registered 'FertileStatsResourceHandleOp' in binary running on DESKTOP-4T2DMPN. Make sure the Op and Kernel are registered in the binary running in this process.